### 1. Import library

In [1]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import pandas as pd

### 2. Crawl data q_a

##### 2.1. Find elements

In [3]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })

    return articles_data

In [4]:
articles_data = []

for page_num in range(1, 301):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/giao-duc-dao-tao-y-te/t1/15/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

##### 2.3 Export CSV

In [5]:
# Tạo DataFrame từ danh sách dữ liệu
df_giaoduc_daotao_yte = pd.DataFrame(articles_data)
df_giaoduc_daotao_yte.head(5)

,Name,Time,Question,Situation,Answer
0,Nguyễn Thị Hài,08:12 08/08/2024,Trường hợp nào được tạm hoãn gọi nhập ngũ?,"Tôi xin hỏi, học sinh vừa tốt nghiệp THPT khi ...",Bộ Quốc phòng trả lời vấn đề này như sau:Theo ...
1,Trần Việt Hưng,08:14 06/08/2024,Tra cứu thông tin cá nhân đủ điều kiện tư vấn ...,Hiện nay khi tra cứu trên Hệ thống thông tin g...,Bộ Y tế trả lời vấn đề này như sau:Thực hiện t...
2,Trần Thị Hoài,09:05 01/08/2024,Có được sử dụng đồng thời chứng chỉ hành nghề ...,"Tôi tốt nghiệp cao đẳng dược, đã có chứng chỉ ...",Bộ Y tế trả lời vấn đề này như sau:TạiLuật Dượ...
3,Nguyễn Minh Hiếu,08:09 01/08/2024,Chưa chuyển mã ngạch có được hưởng phụ cấp nghề?,"Tôi là kỹ sư an toàn thực phẩm, làm việc tại K...","Bộ Y tế trả lời vấn đề này như sau:Hiện nay, c..."
4,Nguyễn Thị Hiếu,10:05 31/07/2024,Viên chức khoa học và công nghệ có được phụ cấ...,"Tôi là kỹ sư công nghệ thực phẩm, mã ngạch 13....","Bộ Y tế trả lời vấn đề này như sau:Hiện nay, c..."


In [6]:
# Lưu DataFrame vào file CSV
csv_file = '../data/giaoduc_daotao_yte_data.csv'
df_giaoduc_daotao_yte.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../data/giaoduc_daotao_yte_data.csv


In [3]:
df_giaoduc_daotao_yte=pd.read_csv("../data/giaoduc_daotao_yte_data.csv",encoding='utf-8')

In [4]:
df_giaoduc_daotao_yte.head(5)

,Index,Name,Time,Question,Situation,Answer
0,0,Nguyễn Thị Hài,08:12 08/08/2024,Trường hợp nào được tạm hoãn gọi nhập ngũ?,"Tôi xin hỏi, học sinh vừa tốt nghiệp THPT khi ...",Bộ Quốc phòng trả lời vấn đề này như sau:Theo ...
1,1,Trần Việt Hưng,08:14 06/08/2024,Tra cứu thông tin cá nhân đủ điều kiện tư vấn ...,Hiện nay khi tra cứu trên Hệ thống thông tin g...,Bộ Y tế trả lời vấn đề này như sau:Thực hiện t...
2,2,Trần Thị Hoài,09:05 01/08/2024,Có được sử dụng đồng thời chứng chỉ hành nghề ...,"Tôi tốt nghiệp cao đẳng dược, đã có chứng chỉ ...",Bộ Y tế trả lời vấn đề này như sau:TạiLuật Dượ...
3,3,Nguyễn Minh Hiếu,08:09 01/08/2024,Chưa chuyển mã ngạch có được hưởng phụ cấp nghề?,"Tôi là kỹ sư an toàn thực phẩm, làm việc tại K...","Bộ Y tế trả lời vấn đề này như sau:Hiện nay, c..."
4,4,Nguyễn Thị Hiếu,10:05 31/07/2024,Viên chức khoa học và công nghệ có được phụ cấ...,"Tôi là kỹ sư công nghệ thực phẩm, mã ngạch 13....","Bộ Y tế trả lời vấn đề này như sau:Hiện nay, c..."


##### 2.4 Export PDF

In [5]:
df_selected = df_giaoduc_daotao_yte[['Question', 'Situation', 'Answer']]

# Tạo lớp kế thừa từ FPDF để tạo PDF
class PDF(FPDF):
    def chapter_title(self, title):
        self.set_font('DejaVuSans-Bold', '', 12)
        self.cell(0, 10, title, 0, 1, 'L')
    
    def chapter_body(self, body):
        self.set_font('DejaVuSans', '', 12)
        self.multi_cell(0, 10, body)
    
    def add_question(self, question, situation, answer):
        self.add_page()
        self.chapter_title('Question:')
        self.chapter_body(question)
        self.chapter_title('Situation:')
        self.chapter_body(situation)
        self.chapter_title('Answer:')
        self.chapter_body(answer)

In [6]:
# Tạo đối tượng PDF
pdf = PDF()
pdf.add_font('DejaVuSans', '', 'DejaVuSans.ttf', uni=True)  # Thêm font Unicode
pdf.add_font('DejaVuSans-Bold', '', 'dejavu-sans-bold.ttf', uni=True)  # Thêm font Unicode
pdf.set_left_margin(10)
pdf.set_right_margin(10)

# Thêm dữ liệu vào PDF
for index, row in df_selected.iterrows():
    pdf.add_question(row['Question'], row['Situation'], row['Answer'])

# Lưu file PDF
pdf.output('../data/giaoduc_daotao_yte_2.pdf', 'F')

RuntimeError: TTF Font file not found: DejaVuSans.ttf